In [2]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import svm
from sklearn.preprocessing import scale
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, train_test_split

import hervpd as hp

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
df = pd.read_csv( 'data/12features/combined.csv')
print(df.activity.unique())
df.describe()


['eat' 'focused-active' 'focused-passive' 'household-chores'
 'leisure-active' 'leisure-passive' 'movement' 'sleep' 'rest-active']


,user,AVGNN,SDNN,MeanHR,MinHR,MaxHR,RMSSD,pNNxx,TINN,powerVLF,powerLF,powerHF,ratioHFLF
count,544.000000,544.000000,544.000000,544.00000,544.000000,544.000000,544.000000,544.000000,544.000000,544.000000,544.000000,544.000000,544.000000
mean,1.187500,758.600000,72.865018,81.37714,67.362033,108.560417,46.409227,20.082227,358.020221,2877.387599,1423.919671,1000.824001,2.753764
std,0.913249,129.785003,32.960406,13.75457,10.849680,36.385572,32.537407,18.316263,146.885899,3619.576847,1443.217366,1841.656665,2.296974
min,0.000000,475.730000,17.833000,51.62800,41.437000,58.117000,4.882500,0.000000,83.000000,85.654000,57.852000,7.977700,0.154680
25%,0.000000,670.362500,51.871500,73.23225,60.292750,91.470250,27.438750,6.593075,258.000000,995.182500,591.307500,258.917500,1.257000
50%,2.000000,750.545000,67.057000,79.94200,66.181500,100.520000,37.994500,15.417500,339.000000,1807.950000,1040.350000,512.090000,2.037350
75%,2.000000,819.307500,85.820000,89.50400,74.285250,112.580000,54.762250,26.790500,426.250000,3304.250000,1697.950000,960.045000,3.579975
max,2.000000,1162.200000,264.250000,126.12000,104.900000,550.320000,240.470000,98.232000,1103.000000,29716.000000,12958.000000,16103.000000,18.228000


### Apply scale to make all features go to mean = 0 and SD = 1. That will allow SVM kernels not to put too much weight on features with larger values.

In [ ]:
#df.iloc[:, 3:15] = df.iloc[:, 3:15].apply(lambda x: scale(x))
df = hp.scaleWithinUser(df)
df.describe()

### Test dataset will have 20% of examples, the other 80% will be for training (with cross-validation)

In [ ]:
train, test = train_test_split(df, test_size=0.2)
print (len(train), len(test))

### 3 - Model selection:  Perform an exhaustive search, with k-fold cross-validation,  in the space parameter consisting of:
* $C = 10^{i}, com -1 \leq i \leq 6$, for both linear and RBF kernels
* $\gamma = 10^{i}, com -3 \leq i \leq 3$, for RBF kernel.

In [ ]:
crossval = StratifiedShuffleSplit(n_splits=4, test_size=0.2)

c_range = np.logspace(-1, 2, 4) 
gamma_range = np.logspace(-2, 1, 4)

param_lin=dict(C=c_range)
param_rbf=dict(C=c_range, gamma=gamma_range)


In [ ]:
grid_lin = GridSearchCV(svm.SVC(kernel='linear', cache_size=1000), param_grid=param_lin, cv=crossval)
grid_lin.fit(X=train.iloc[:, 3:15], y=train['activity'])

print("Kernel linear --- ")
print("Best params: %s with score %0.5f" % (grid_lin.best_params_, grid_lin.best_score_))

In [ ]:
grid_rbf = GridSearchCV(svm.SVC(kernel='rbf', cache_size=1000), param_grid=param_rbf, cv=crossval)
grid_rbf.fit(X=train.iloc[:, 3:15], y=train['activity'])

print("Kernel RBF --- ")
print("Best params: %s with score %0.5f" % (grid_rbf.best_params_, grid_rbf.best_score_))

In [ ]:
clf1 = svm.SVC(kernel='linear', cache_size=1000, C=grid_lin.best_params_['C'])
clf1.fit(X=train.iloc[:, 3:15], y=train['activity'])
hp.printResults(test['activity'].values, clf1.predict(test.iloc[:, 3:15]))

clf2 = svm.SVC(kernel='rbf', cache_size=1000, C=grid_rbf.best_params_['C'], gamma=grid_rbf.best_params_['gamma'])
clf2.fit(X=train.iloc[:, 3:15], y=train['activity'])
hp.printResults(test['activity'].values, clf1.predict(test.iloc[:, 3:15]))

## Now, we can run the same steps above with multiple experiments! Time to have fun!!

In [23]:
df = pd.read_csv( 'data/12features/combined.csv')
hp.runFlow(df)
df.describe()

C:\Users\julia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
C:\Users\julia\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


435 train examples and 109 test examples
Best params for linear kernel: {'C': 1.0} with score 0.45977
Best params for RBF kernel: {'C': 10.0, 'gamma': 0.10000000000000001} with score 0.43103
--- test results for linear kernel:
39 out of 109 right! :)
--- test results for RBF kernel:
39 out of 109 right! :)


### 1 - All activities per user

In [ ]:
ju = hp.userRows(df, 0)
ron = hp.userRows(df, 1)
edu = hp.userRows(df, 2)

hp.runFlow(ju)
hp.runFlow(ron)
hp.runFlow(edu)

### 2 - Splitting database into movement categories (movement, household chores and exercise) and all others

In [31]:
includelist = ['movement', 'household-chores']

dfmove = hp.addPartition(df, includelist, pname='move', labelIn='move', labelOut='still')
hp.runFlow(dfmove, labelName='move')

C:\Users\julia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
C:\Users\julia\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


435 train examples and 109 test examples
Best params for linear kernel: {'C': 0.10000000000000001} with score 0.90230
Best params for RBF kernel: {'C': 1.0, 'gamma': 0.01} with score 0.88218
--- test results for linear kernel:
96 out of 109 right! :)
--- test results for RBF kernel:
96 out of 109 right! :)


In [32]:
jumove = hp.addPartition(ju, includelist, pname='move', labelIn='move', labelOut='still')
hp.runFlow(jumove, labelName='move')

C:\Users\julia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


148 train examples and 37 test examples
Best params for linear kernel: {'C': 0.10000000000000001} with score 0.88333
Best params for RBF kernel: {'C': 1.0, 'gamma': 0.01} with score 0.90833
--- test results for linear kernel:
30 out of 37 right! :)
--- test results for RBF kernel:
30 out of 37 right! :)


In [34]:
edumove = hp.addPartition(edu, includelist, pname='move', labelIn='move', labelOut='still')
hp.runFlow(edumove, labelName='move')

C:\Users\julia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


229 train examples and 58 test examples
Best params for linear kernel: {'C': 10.0} with score 0.86413
Best params for RBF kernel: {'C': 10.0, 'gamma': 0.10000000000000001} with score 0.89674
--- test results for linear kernel:
49 out of 58 right! :)
--- test results for RBF kernel:
49 out of 58 right! :)


In [35]:
ronmove = hp.addPartition(ron, includelist, pname='move', labelIn='move', labelOut='still')
hp.runFlow(ronmove, labelName='move')

C:\Users\julia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


57 train examples and 15 test examples
Best params for linear kernel: {'C': 0.10000000000000001} with score 1.00000
Best params for RBF kernel: {'C': 1.0, 'gamma': 0.01} with score 0.97917
--- test results for linear kernel:
15 out of 15 right! :)
--- test results for RBF kernel:
15 out of 15 right! :)


In [37]:
dfnomove = hp.excludeActivities(df, includelist)
dfnomove.describe()
hp.runFlow(dfnomove)

C:\Users\julia\Anaconda3\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int64 was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
C:\Users\julia\Anaconda3\lib\site-packages\pandas\core\indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


328 train examples and 83 test examples
Best params for linear kernel: {'C': 0.10000000000000001} with score 0.43182
Best params for RBF kernel: {'C': 100.0, 'gamma': 0.10000000000000001} with score 0.42424
--- test results for linear kernel:
34 out of 83 right! :)
--- test results for RBF kernel:
34 out of 83 right! :)
